In [397]:
import numpy as np
import cv2
import colorsys

In [398]:
def extract_ring_2(path):
    # normalize image
    og_img = cv2.imread(path)
    B, G, R = cv2.split(og_img)
    B_norm = cv2.normalize(B, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)
    G_norm = cv2.normalize(G, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)
    R_norm = cv2.normalize(R, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)
    normalized_image = cv2.merge([B_norm, G_norm, R_norm])
    img = np.uint8(normalized_image)

    # saturate it to pump up R and B channels while decreasing G channel for magenta
    hsv_image = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv_image)
    s = s.astype(np.float32)
    s = s * 1.5
    v = v * 1.5
    s = np.clip(s, 0, 255).astype(np.uint8)
    v = np.clip(v, 0, 255).astype(np.uint8)
    hsv_modified = cv2.merge([h, s, v])
    img = cv2.cvtColor(hsv_modified, cv2.COLOR_HSV2BGR)

    # downscale image so less noise
    dimensions = (360, 640)
    scale_factor_x = og_img.shape[1] / dimensions[0]
    scale_factor_y = og_img.shape[0] / dimensions[1]
    img = cv2.resize(img, (dimensions), interpolation=cv2.INTER_AREA)

    # mask out non-magenta colors with looser thresholds
    # then re-normalize so that G channel is even more separate from R and B
    lower2 = np.array([50, 0, 100])     # darken
    upper2 = np.array([255, 150, 255])
    mask2 = cv2.inRange(img, lower2, upper2)
    img = cv2.bitwise_and(img, img, mask=mask2)
    B, G, R = cv2.split(img)
    B_norm = cv2.normalize(B, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)
    G_norm = cv2.normalize(G, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)
    R_norm = cv2.normalize(R, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)
    normalized_image = cv2.merge([B_norm, G_norm, R_norm])
    img = np.uint8(normalized_image)

    # mask out non-magenta colors again with tighter threshold
    lower = np.array([100, 0, 150])
    upper = np.array([255, 200, 255])
    mask = cv2.inRange(img, lower, upper)
    kernel = cv2.getStructuringElement(cv2.MORPH_CROSS, (4, 4))
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel, iterations=2)
    # white_mask = cv2.inRange(img, (180, 180, 180), (255, 255, 255)) # dunno if this helps tbh
    # mask = cv2.subtract(mask, white_mask)
    segmented = cv2.bitwise_and(img, img, mask=mask)
    # cv2.imwrite(f'temp2.png', segmented)

    # find largest magenta contour 
    gray = cv2.cvtColor(segmented, cv2.COLOR_BGR2GRAY)
    contours, _ = cv2.findContours(gray, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    largest_contour = max(contours, key=cv2.contourArea)
    cv2.drawContours(segmented, [largest_contour], -1, (0, 255, 0), 2)
    upscaled_contour = largest_contour.copy().astype(np.float32) # upscale contour to final image
    upscaled_contour[:, :, 0] *= scale_factor_x
    upscaled_contour[:, :, 1] *= scale_factor_y
    upscaled_contour = upscaled_contour.astype(np.int32)

    # and mask the image with that contour
    mask_2 = np.zeros_like(og_img)
    cv2.drawContours(mask_2, [upscaled_contour], -1, (255, 255, 255), thickness=cv2.FILLED)
    white_background = np.full_like(og_img, 255)
    masked_image = np.where(mask_2 == 255, og_img, white_background)

    cv2.imwrite(f'test1.png', masked_image)

extract_ring_2('img_1.jpg', 0)